# geocode() メソッドの理解

arcgis.geocoding モジュールの geocode() メソッドは、リクエストごとに 1 つの場所をジオコーディングします。入力として指定できる住所は、単一行形式の住所文字列か、アドレス フィールドと同じ名前のキーを持つPython の辞書型を使って複数のパラメーターに分割することができます。

<img src="https://developers.arcgis.com/rest/geocode/api-reference/GUID-F2F78690-5FB5-4BF1-8633-26BF184C45A9-web.png"/>

geocode() メソッドは、以下のタイプの場所検索をサポートしています。
* **住所**
 * <code class="usertext">27488 Stanford Ave, Bowden, North Dakota</code>
 * <code class="usertext">380 New York St, Redlands, CA 92373</code>
* **POI (Points of interest)** ※[POI とは](https://www.esrij.com/gis-guide/gis-other/point-of-interest/)
 * <code class="usertext">Disneyland</code>
 * <code class="usertext">banks in Paris</code>
 * <code class="usertext">los angeles starbucks</code>
 * <code class="usertext">mount everest</code>
* **市名、郡名、州名、県名、国名などの行政地名**
 * <code class="usertext">Seattle, Washington</code>
 * <code class="usertext">State of Mahārāshtra</code>
 * <code class="usertext">Liechtenstein</code>
 * <code class="usertext">mount everest</code>
* **郵便番号**
 * <code class="usertext">92591</code>
 * <code class="usertext">TW9 1DN</code>

検索結果の絞り込みには次のようないくつかのオプションがあります。
* ジオコーディングで返す出力フィールドを out_fields パラメーターで指定します。.
* 空間参照を out_sr パラメータで指定します。.
* max_locations パラメーターで候補の数を制限します。
* search_extent パラメーターで検索結果を指定した領域に限定します。
* location パラメーターや distance パラメーターを使用して、ジオコーディングの候補をその位置の近接度をもとにソートします。
* category パラメーターを使用して検索結果をフィルタリングします。

このノートブックでは 次の geocode()メソッドのパラメーターについてご紹介します。
 - [`geocode()` メソッドのパラメーター](#geocode-function-signature-and-parameters)
   - [`address` パラメーター](#address-parameter)
     - [single line address](#single-line-address)
     - [multi field address](#multi-field-address)
   - [`search_extent` パラメーター](#search-extent-parameter)
   - [`location` パラメーター](#location-parameter)
   - [`distance` パラメーター](#distance-parameter)
   - [`category` パラメーター](#category-parameter)
   - [`out_sr` パラメーター](#out-sr-parameter)
   - [`out_fields` パラメーター](#out-fields-parameter)
   - [`max_locations` パラメーター](#max-locations-parameter)
   - [`for_storage` パラメーター](#for-storage-parameter)

<a id="geocode-function-signature-and-parameters"></a>
## geocode() メソッドのパラメーター
geocode()メソッドは、単一フィールド形式での場所や住所の検索や、複数のパラメーターに住所を分割した複数フィールド形式での検索をサポートしています。

以下では、geocode()メソッドの各パラメーターについてを簡単ご紹介します。

In [1]:
from arcgis.geocoding import geocode

In [2]:
# メソッドのヘルプを表示することができます
help(geocode)

Help on function geocode in module arcgis.geocoding._functions:

geocode(address, search_extent=None, location=None, distance=None, out_sr=None, category=None, out_fields='*', max_locations=20, magic_key=None, for_storage=False, geocoder=None, as_featureset=False)
    The geocode function geocodes one location per request.
    
    ===============     ====================================================
    **Argument**        **Description**
    ---------------     ----------------------------------------------------
    address             required list of strings or dictionaries.
                        Specifies the location to be geocoded. This can be
                        a string containing the street address, place name,
                        postal code, or POI.
    
                        Alternatively, this can be a dictionary containing
                        the various address fields accepted by the
                        corresponding geocoder. These fields are li

<a id="address-parameter"></a>
### address パラメーター
<a id="single-line-address"></a>
#### 単一行の住所

address パラメーターにはジオコーディングする住所を指定します。単一行の住所や地名、郵便番号、または POI を含む文字列を指定することができます。

#### 例 : 単一行の住所を指定したジオコーディング

In [16]:
from arcgis.gis import GIS

gis = GIS()
map = gis.map("東京都千代田区平河町")
map

MapView(layout=Layout(height='400px', width='100%'))

In [17]:
single_line_address = "東京都千代田区平河町2-7-1"

In [18]:
# 単一行の住所をジオコーディングし、結果の1番目を地図上にプロットします。
esrij = geocode(single_line_address)[0]

# シンボル設定用の辞書 (dictionary) 生成ツール: https://esri.github.io/arcgis-python-api/tools/symbol.html
symbol = {"color":[0,0,128,128],
          "size":18,
          "type":"esriSMS",
          "style":"esriSMSCircle",
          "outline":{"color":[0,0,128,255],
                     "width":1,
                     "type":"esriSLS",
                     "style":"esriSLSSolid"}}

# ポップアップを追加します
popup = { 
    "title" : "ESRIジャパン株式会社", 
    "content" : esrij['address']
    }
map.draw(esrij['location'], popup, symbol=symbol)

In [19]:
# 結果の場所(x,y)を表示します。
esrij['location']

{'x': 139.740651,
 'y': 35.679213000000004,
 'type': 'point',
 'spatialReference': {'wkid': 4326}}

<a id="multi-field-address"></a>
#### 複数フィールドの住所
geocode() メソッドは、ジオコード サービスで受け入れ可能なフィールド名に対応した辞書（Dict）を使用して、複数フィールドの形式で住所を指定することもできます。

また、多くの異なる国々の住所情報に対応するため、geocode() メソッドは標準化されたフィールド名を使用します。

ジオコーダーの addressFields プロパティは、アドレスをジオコーディングする際に受け入れ可能な様々なアドレスフィールドを指定します。neighborhood, city, subregion, region の各パラメーターは、国の行政区分を表します。これらは国によって異なり、すべての国ですべての行政区分が使用されているわけではありません。例えば、アメリカの住所では、市(city)と地域(state)のパラメーターのみが使用されます。メキシコの住所では、市の中の地区(colonias)、市の中の市町村(municipios)、州(estados)には地域パラメータが使用され、スペインでは4つの行政区分すべてが使用されます。

例えば、ジオコードサービスの addressFields に Address、City、Region、Postalという名前のフィールドが含まれている場合、以下のような形式になります。

#### 例: 複数フィールドの住所を指定したジオコーディング

In [23]:
multi_field_address = { 
    "Address" : "平河町2-7-1",
    "City" : "千代田区",
    "Region" : "東京都",
    "Postal" : 1020093
    }

In [24]:
map = gis.map("東京都千代田区")
map

MapView(layout=Layout(height='400px', width='100%'))

In [25]:
# 複数フィールドの住所をジオコーディングし、結果の１番目を地図上にプロットします。

esrij = geocode(multi_field_address)[0]

# シンボル設定用の辞書 (dictionary) 生成ツール: https://esri.github.io/arcgis-python-api/tools/symbol.html
symbol = {"color":[0,0,128,128],
          "size":18,
          "type":"esriSMS",
          "style":"esriSMSCircle",
          "outline":{"color":[0,0,128,255],
                     "width":1,
                     "type":"esriSLS",
                     "style":"esriSLSSolid"}}

popup = { 
    "title" : "ESRIジャパン株式会社", 
    "content" : esrij['address']
    }
map.draw(esrij['location'], popup, symbol=symbol)

geocode() メソッドは、一致する可能性のあるアドレスの配列 (アドレス候補) を返します。各アドレス候補は、以下のキーを持つ Python の辞書型として表現されます。

In [26]:
esrij.keys()

dict_keys(['address', 'location', 'score', 'attributes', 'extent'])

辞書のキーの値は以下の通りです。

* <code>score</code> ：入力した住所が返却された候補の住所情報とどの程度マッチしているかを 0～100 の値で表します。100 は完全に一致していることを表し、低い値は一致度合いが低いことを表します。
* <code>location</code>：マッチした住所の位置情報 (x, y) が含まれています。
* <code>address</code> ：マッチした住所が含まれています。
* <code>attributes</code> ：上記のアドレス候補 プロパティに関連した幾つかのパラメーターが含まれています。
* <code>extent</code> ：マッチした住所のエクステントが含まれています。

<a id="search-extent-parameter"></a>
### search_extent パラメーター

検索範囲を特定の地域に限定する座標を指定することが可能です。これは、特にユーザーが現在のマップの表示範囲内で、場所や住所を検索するアプリケーションでの利用する場合に便利です。

search_extent で座標の空間参照を指定することが可能ですが、これは地図の空間参照がジオコーディング サービスの空間参照と異なる場合に必要で、それ以外は、座標の空間参照がジオコーディング サービスの空間参照と同一であると見なされます。

#### 例：千代田区平河町周辺の病院

以下の例では、千代田区平河町周辺にある病院を検索してプロットしています。
search_extent パラメーターには、東京都千代田区平河町のジオコーディング結果 (result['extent']) から取得したエクステントを渡しています。

In [27]:
result = geocode("東京都千代田区平河町")[0]
map = gis.map(result)
map

MapView(layout=Layout(height='400px', width='100%'))

In [28]:
result['extent']

{'xmin': 139.7319804050001,
 'ymin': 35.67275705600006,
 'xmax': 139.7479804050001,
 'ymax': 35.688757056000064}

In [29]:
# 平河町周辺の病院をジオコーディング
# ArcGIS World Geocoding Service を用いて郵便局、病院など不特定のPOIを検索する際は英単語で行うことをおすすめします。
hospitals = geocode("hospital", result['extent'], max_locations=100)

# シンボル設定用の辞書 (dictionary) 生成ツール: https://esri.github.io/arcgis-python-api/tools/symbol.html
symbol = {"color":[0,0,128,128],
          "size":18,
          "type":"esriSMS",
          "style":"esriSMSCircle",
          "outline":{"color":[0,0,128,255],
                     "width":1,
                     "type":"esriSLS",
                     "style":"esriSLSSolid"}}

for hospital in hospitals:
    map.draw(hospital['location'] ,symbol=symbol)

<a id="location-parameter"></a>
### location パラメーター

<code>
※ 本ノートブックの翻訳時点では、ArcGIS API for Python の API リファレンスの location パラメーターの説明では、 distance パラメーターと併せて用いるものとして説明されていますが、ラッピングしている ArcGIS REST API のドキュメントでは distance パラメーターは既に廃止されており、パラメーターを渡しても認識されない旨が記載されています。
</code>

location パラメーターでは近さに基づいてジオコーディングの候補をソートするために、distance パラメーターと共に使用されます。distance パラメーターは、その場所からの半径をメートル単位で指定します。この半径内の候補の優先度は、半径外の候補よりも相対的に高くなります。

これは、現在のGPS位置情報から、近くにある場所を検索するモバイルアプリケーションでの利用に便利な機能です。

特定の距離をlocation パラメーターで指定しない場合、デフォルトで 50000 メートル が使用されます。

location パラメーターは、シンプルなカンマ区切りでの(x,y)のように表すか、JSON ポイント オブジェクトとして表して、指定することが可能です。カンマ区切りでの(x,y) で指定する場合、空間参照は WGS84 でなければなりません。それ以外の場合は、ポイントの座標の空間参照をJSON オブジェクトとして指定することが可能です。


・シンプルな指定（WGS84）の例:
<code>
location=-117.196,34.056
</code>

・ポイントの座標の空間参照を使ってJSON ポイントオブジェクトで指定した例：
<code>
location=
{
    "x": -13046165.572,
    "y": 4036389.847,
    "spatialReference": {
        "wkid": 102100
    }
}
</code>

<a id="distance-parameter"></a>
### distance パラメーター

distance パラメーターはジオコーディングの候補のランクを上げるために使用するパラメーターで、点の位置からの周囲の半径をメートル単位で指定します。

distance パラメーターを指定する場合は、location パラメーターも指定する必要があります。

search_extent パラメーターとは異なり、location パラメーターと distance パラメーターは、指定した検索半径を超えて検索することに注意が必要です。これらは、結果をフィルタリングするために使用されるのではなく、指定した場所からの距離に基づいた候補のランク付けを行うために使用されます。検索結果を特定のエリアに限定したい場合は、location と distance に加えて search_extent パラメーターを指定する必要があります。

・現在の範囲の 3 km 以内で検索した例:
<code>
distance=3000
</code>

<a id="category-parameter"></a>
### category パラメーター

category パラメーターはジオコーディングの結果をフィルタリングするために使用できる場所またはアドレスのタイプを指定することができます。このパラメーターは、1つのカテゴリの値、もしくはカンマ区切りでの複数のカテゴリの値をサポートします。category パラメーターは、1つのカテゴリ、複数のカテゴリでも、1つのリクエストに含めて渡すことが可能です。

・1つのカテゴリでフィルタリングを行う例:
<code>
category="Address"
</code>

・複数のカテゴリでフィルタリングを行う例:
<code>
category="Address,Postal"
</code>

category パラメーターは、単一フィールド形式での利用の場合のみ機能し、複数フィールド形式での利用は動作しません。
カテゴリの詳細は、<a href="https://developers.arcgis.com/rest/geocode/api-reference/geocoding-category-filtering.htm">Category filtering</a> をご参照ください。


### 例:千代田区平河町周辺の飲食店を検索

In [34]:
result = geocode("東京都千代田区平河町")[0]

map = gis.map(result)
map

MapView(layout=Layout(height='400px', width='100%'))

In [35]:
# 東京都千代田区平河町周辺の飲食店を検索しプロット

restaurants = geocode(None, result['extent'], category="Food", max_locations=100)

# シンボル設定用の辞書 (dictionary) 生成ツール: https://esri.github.io/arcgis-python-api/tools/symbol.html
symbol = {"color":[0,0,128,128],
          "size":18,
          "type":"esriSMS",
          "style":"esriSMSCircle",
          "outline":{"color":[0,0,128,255],
                     "width":1,
                     "type":"esriSLS",
                     "style":"esriSLSSolid"}}

for restaurant in restaurants:
    popup = { 
    "title" : "平河町周辺飲食店", 
    "content" : restaurant['address']
    }
    map.draw(restaurant['location'], popup, symbol=symbol)

### 例:複数のカテゴリで検索し、異なるシンボルでプロット

以下の例では、平河町周辺での飲食店とコンビニエンスストアを検索し、'Type'属性に基づいて異なるシンボルを使用してそれらの場所をプロットしています。

In [36]:
categories = "Food, Convenience Store"

In [37]:
result = geocode("東京都千代田区平河町")[0]

map = gis.map(result)
map

MapView(layout=Layout(height='400px', width='100%'))

In [38]:
# 平河町周辺での飲食店とコンビニエンスストアを検索

# シンボル設定用の辞書 (dictionary) 生成ツール: https://esri.github.io/arcgis-python-api/tools/symbol.html
symbol = {"color":[0,0,128,128],
          "size":18,
          "type":"esriSMS",
          "style":"esriSMSCircle",
          "outline":{"color":[0,0,128,255],
                     "width":1,
                     "type":"esriSLS",
                     "style":"esriSLSSolid"}}

symbol2 = {"type": "esriSMS",
           "style": "esriSMSSquare",
           "color": [76,115,0,255],
           "size": 8,
           "angle": 0,
           "xoffset": 0,
           "yoffset": 0,
           "outline":{"color": [152,230,0,255],
                      "width": 1}}

restaurants = geocode(None,result['extent'], category=categories, max_locations=100)
for restaurant in restaurants:
    popup = { 
    "title" : restaurant['address'], 
    "content" : "Phone: " + restaurant['attributes']['Phone']
        }
    if restaurant['attributes']['Type'] == 'Convenience Store':
        map.draw(restaurant['location'], popup, symbol2) # 異なる（緑色の四角形）シンボルで描画
    else:
        map.draw(restaurant['location'], popup, symbol)

<a id="out-sr-parameter"></a>
### out_sr  パラメーター

geocode() メソッドが返す xy 座標の空間参照を指定することができます。これは、ジオコーダーの空間参照とは異なる空間参照を持つ地図を使用するアプリケーションでの利用する場合に便利です。

空間参照は、Well Known ID（WKID）または JSON の空間参照オブジェクトのいずれかで指定することができます。out_sr パラメーターが指定されていない場合、空間参照はジオコーダーの空間参照と同じになります。World Geocoding Service の空間参照は WGS84（WKID = 4326）です。

WKID については、<a href="https://developers.arcgis.com/rest/services-reference/projected-coordinate-systems.htm">Projected Coordinate Systems</a> や <a href="https://developers.arcgis.com/rest/services-reference/geographic-coordinate-systems.htm">Geographic Coordinate Systems</a> を参照してください。

・Web メルカトル座標系の WKID (102100)を指定した例:
<code>
out_sr=102100
</code>

<a id="out-fields-parameter"></a>
### out_fields パラメーター

返却されるフィールドのリストです。各フィールドについては、<a href="https://developers.arcgis.com/rest/geocode/api-reference/geocoding-service-output.htm#ESRI_SECTION1_42D7D3D0231241E9B656C01438209440">Output fields</a> を参照してください。

デフォルトでは、マッチした住所とXY 座標、マッチスコア、空間参照、出力フィーチャの範囲、および addr_type (マッチレベル) が返されます。

・すべての出力フィールドを返す例:
<code>
out_fields=*
</code>

・指定したフィールドのみを返す例:
<code>
out_fields="AddrNum,StName,City"
</code>

<a id="max-locations-parameter"></a>
### max_locations パラメーター

検索によって返される場所の最大数を指定します。指定されていない場合は、最大数までのすべてのマッチする候補が返されます。

World Geocoding Service は、最大20個の候補を返すことができます。

・例:
<code>
max_locations=10
</code>

<a id="for-storage-parameter"></a>
### for_storage パラメーター

ジオコーディング結果を保存するかどうかを指定します。デフォルトは false で、保存することはできませんが、一時的にマップなどに表示することはできます。結果をデータベースなどに保存する場合は、このパラメーターを true に設定する必要があります。

for_storage パラメーターに true の値を指定してリクエストを行わない限り、ジオコーディングトランザクションの結果を保存することは契約上禁止されています。

ArcGIS Online サービス クレジットは、値が true の for_storage パラメーターを含むジオコード トランザクションごとに組織アカウントから差し引かれます。クレジットの詳細については、<a href="https://www.esri.com/en-us/arcgis/products/credits/overview?rmedium=esri_com_redirects01&rsource=https://www.esri.com/en-us/arcgis/products/arcgis-online/pricing/credits">ArcGIS Online Credits (英語)</a> や<a href="https://www.esrij.com/products/arcgis-online/credits/">クレジット</a>を参照してください。


無料/有料でのジオコーディング操作の詳細については、<a href="https://developers.arcgis.com/rest/geocode/api-reference/geocoding-free-vs-paid.htm">Free vs. paid operations</a> を参照してください。

・例:
<code>
for_storage=True
</code>